## Import libraries

In [1]:
import requests
import pandas as pd
from prophet import Prophet
from datetime import datetime

e:\Github\Dhaka-Weather-Forecasting\mlapi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


## Collecting data using weather API

Using the API I am collecting data from 01 Jan 2023 to 31 May 2023. Here only temperature data is collecting at 120 meters above ground. The website can be found [here.](https://open-meteo.com/en/docs#latitude=23.71&longitude=90.41&hourly=temperature_2m)

In [2]:
# API endpoint and parameters
api_url = "https://api.open-meteo.com/v1/forecast"
latitude = 23.71
longitude = 90.41
start_date = "2023-01-01"
end_date = "2023-05-31"
parameters = "temperature_120m"  # hourly temperature data at 120 meters above ground

# Construct the API request URL
url = f"{api_url}?latitude={latitude}&longitude={longitude}&hourly={parameters}&start_date={start_date}&end_date={end_date}"
# url = "https://api.open-meteo.com/v1/forecast?latitude=23.71&longitude=90.41&hourly=temperature_120m&start_date=2022-06-08&end_date=2023-05-31"

# Make the API request
response = requests.get(url)

In [3]:
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)

else:
    print("Failed to fetch weather data. Status code:", response.status_code)

In [4]:
df.head().T     # showing collected data.

,time,temperature_120m
latitude,23.75,23.75
longitude,90.375,90.375
generationtime_ms,43.970942,43.970942
utc_offset_seconds,0,0
timezone,GMT,GMT
timezone_abbreviation,GMT,GMT
elevation,16.0,16.0
hourly_units,iso8601,°C
hourly,"[2023-01-01T00:00, 2023-01-01T01:00, 2023-01-0...","[16.5, 15.7, 15.2, 13.9, 14.8, 17.2, 17.9, 19...."


## Data pre-processing

The input to Prophet is always a dataframe with two columns: ds and y. The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric, and represents the measurement we wish to forecast.

In [5]:
new_df = df["hourly"]
new_df.head()

time                [2023-01-01T00:00, 2023-01-01T01:00, 2023-01-0...
temperature_120m    [16.5, 15.7, 15.2, 13.9, 14.8, 17.2, 17.9, 19....
Name: hourly, dtype: object

In [6]:
dataset = {
    "ds" : new_df["time"],
    "y" : new_df["temperature_120m"]
}

dataset = pd.DataFrame.from_dict(dataset)
print(dataset.shape)
print(dataset.head())

(3624, 2)
                 ds     y
0  2023-01-01T00:00  16.5
1  2023-01-01T01:00  15.7
2  2023-01-01T02:00  15.2
3  2023-01-01T03:00  13.9
4  2023-01-01T04:00  14.8


Changing the date formate

In [7]:
def convert_date(date_str):
  '''
  Change string date to YYYY-MM-DD HH:MM:SS format

  Parameters
  -------------
  date_str (string): date as string
  
  Return:
  -------------
  return YYYY-MM-DD HH:MM:SS format date
  '''
  datetime_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M")
  formatted_date = datetime_obj.strftime("%Y-%m-%d %H:%M:%S")
  return formatted_date

In [8]:
dataset["ds"] = dataset["ds"].apply(convert_date)
print(dataset.shape)
print(dataset.head())

(3624, 2)
                    ds     y
0  2023-01-01 00:00:00  16.5
1  2023-01-01 01:00:00  15.7
2  2023-01-01 02:00:00  15.2
3  2023-01-01 03:00:00  13.9
4  2023-01-01 04:00:00  14.8


## Training Model

Here I used a famous forecasting model named prophet. Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

Prophet is open source software released by Facebook's Core Data Science team. It is available for download on CRAN and PyPI. The models documentation can be found [here.](https://facebook.github.io/prophet/)

In [9]:
model = Prophet()
model.fit(dataset)

01:19:30 - cmdstanpy - INFO - Chain [1] start processing
01:19:30 - cmdstanpy - INFO - Chain [1] done processing


## Prediction

In [10]:
predict_date = '2024-01-01 04:00:00'
future = list()
future.append([predict_date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds'] = pd.to_datetime(future['ds'])
print(future.head())

                   ds
0 2024-01-01 04:00:00


In [11]:
forecast = model.predict(pd.DataFrame.from_dict(future))
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2024-01-01 04:00:00,25.343917,23.150401,27.656345


## Save model


In [15]:
from prophet.serialize import model_to_json

with open('model.json', 'w') as fout:
    fout.write(model_to_json(model))